In [1]:
pip install opencv-python     #Installing opencv

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2      #importing opencv
import numpy as np   #import numpy

In [2]:
cap = cv2.VideoCapture('apples.mp4')    #Open the video file
if cap.isOpened==False :                #Checks if video was opened correctly or not
    print("Error opening video stream or file")

# Method 1

The target is to count the number of apples in a video. We shall first detect the apples in each frame by looking at the contours in each blurred frame. Then, count the number of apples as they go accross a vertical line drawn in the video. 

In [3]:
bgsub = cv2.createBackgroundSubtractorMOG2()  #creates an object to subtract the background

In [4]:
def object_center(x,y,w,h):  #calculates the center of an apple
    x1 = int(w/2)
    y1 = int(h/2)
    cx = x+x1
    cy = y+y1
    return cx,cy

In [5]:
line_position = 450   #height of the vertical line
error = 6     #permissible error for the object identification
counter = 0   #count of the apples
min_width = 10   #min width of the rectangle
min_height = 10   #min height of the rectangle
detect = []  #list to store the center of an apple

In [6]:
while cap.isOpened():   #When the video is opened
    ret,frame = cap.read()    #returns each frame of the video
    
    if ret==True:     #Checks frame is being returned or not
        img_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)   #changes the color space of image to grayscale
        img_blur = cv2.GaussianBlur(img_gray,(3,3),5)    #blurs the image
        img_bgsub = bgsub.apply(img_blur)     #removes the background from the image
        img_dil = cv2.dilate(img_bgsub,np.ones((5,5)))  #widens the image
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))  #to get an elliptical kernel
        img_close = cv2.morphologyEx(img_dil, cv2.MORPH_CLOSE,kernel)  #perfoms closing of the image using the kernel
        img_close = cv2.morphologyEx(img_close, cv2.MORPH_CLOSE,kernel)
        img_contour,h = cv2.findContours(img_close, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #finds all the contours in the image

        cv2.line(frame, (15,line_position),(700,line_position), (255,0,0),2)  #adds a horizontal line to each frame
        
        for (i,j) in enumerate(img_contour):
            (x,y,w,h)=cv2.boundingRect(j)  #draws rectangles around the binary images
            max_size = (w>=min_width) and (h>=min_height)
            if not max_size:
                continue
            if y<(line_position):    
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),1)   #creates a rectangle
                center = object_center(x,y,w,h)  #finds the center of each rectangle and hence each apple
                detect.append(center)   
            for (x,y) in detect:   #count the apples as they cross the line
                if y>(line_position-error) and y<(line_position+error):
                    counter+=1
                detect.remove((x,y))
                
                
        cv2.putText(frame,"No. of Apples:"+str(counter),(100,100),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1)  #displays the count
                
    
        cv2.imshow('Apple Counter',frame)
        if cv2.waitKey(1)==13:
            break
    else:
        break
        
        
        
print("Number of Apples: "+str(counter))
cv2.destroyAllWindows()
cap.release()
        

Number of Apples: 25


# Method 2

My attempt was to look at frame in some particular intervals and then detect the number of apples in each such frame. The total number of apples in the video will be equal to the sum of the number of apples over each of these frames. 

In [2]:
cap = cv2.VideoCapture('apples.mp4')    #Open the video file
if cap.isOpened==False :                #Checks if video was opened correctly or not
    print("Error opening video stream or file")

In [3]:
def auto_canny(image, sigma=0.33):
    v = np.median(image)
    lower = int(max(0, (1.0 - sigma) * v))  #lower threshold
    upper = int(min(255, (1.0 + sigma) * v))  #upper threshold
    edged = cv2.Canny(image, lower, upper)  #finds the edges of each frame
    return edged


In [4]:
frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)  #get the number of frames in the video
counter2 = []

In [5]:
frame_count

1095.0

In [ ]:
while cap.isOpened():    #When the video is opened
    for i in range(int(frame_count)):
        ret,frame = cap.read()  #returns each frame of the video
        if i%100==0:
            c_num=0

            if ret==True:    #Checks frame is being returned or not
                img_d = frame.copy()  #create a copy of each frame
                img_blur = cv2.GaussianBlur(frame, (5,5), 0)   #blurs the image
                img_ms = cv2.pyrMeanShiftFiltering(img_blur, 10, 90)  #filters the image

                edge = auto_canny(img_ms)   #finds edges of each frame
                cnts,_ = cv2.findContours(edge.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #Contours of the edged image
                for i,c in enumerate(cnts):
                    ((x, y), r) = cv2.minEnclosingCircle(c) #draw a circle around the objects
                    if r>5:
                        c_num+=1 #increase the count of apples in this frame
                        cv2.circle(img_d, (int(x), int(y)), int(r), (0, 255, 0), 2)   #draw circles around the objects in the original video
                    else:
                        continue
                counter2.append(c_num)  #number of apples in each frame
                cv2.imshow('Original',img_d)
                if cv2.waitKey(1)==13:
                    break
            else:
                break
        
cap.release()
cv2.destroyAllWindows()

print("Number of Apples: "+str(sum(counter2)))  #print the total number of apples
